In [33]:
# load packages
using Gurobi, StatsBase, CSV, DataFrames, 
    JuMP, LinearAlgebra, Distributions, Random,
    GLMNet, ScikitLearn, MLBase, CategoricalArrays, Plots,
    Dates, Clustering, Distances, StatsPlots, ProgressMeter

In [34]:
ProgressMeter.ijulia_behavior(:clear);
const GUROBI_ENV = Gurobi.Env();

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-16


In [59]:
# load data
function load_csv(path)
    data = CSV.read(path, DataFrame) |> Tables.matrix;
   return data
end;

X_train = load_csv("X_train.csv");
y_train = load_csv("y_train.csv");
X_test = load_csv("X_test.csv");
y_test = load_csv("y_test.csv");

In [60]:
size(X_train)

(2041, 191)

## Sparse Logistic Regression

In [66]:
for loss_fn in [:misclassification, :gini, :entropy]
    lnr=IAI.OptimalTreeClassifier(criterion=loss_fn, 
                                    max_depth=3, 
                                    cp=0.001, 
                                    minbucket=5,
                                    random_seed=42)
    display(IAI.fit!(lnr, X_train, y_train))
    
    train_auc_i = [train_auc_i; IAI.score(lnr, X_train, y_train,criterion=:auc)]
    val_auc_i = [val_auc_i; IAI.score(lnr, X_valid, y_valid,criterion=:auc)]
end

MethodError: MethodError: no method matching fit!(::OptimalTrees.OptimalTreeClassifier, ::Matrix{Float64}, ::Matrix{Int64})
Closest candidates are:
  fit!(!Matched::IAIBase.GridSearch{<:IAIBase.SupervisedLearner{<:IAIBase.PolicyTask}}, ::Union{AbstractDataFrame, AbstractMatrix{<:Real}}, ::Union{AbstractDataFrame, AbstractMatrix{<:Real}}; sample_weight, kwargs...) at C:\Users\iai\builds\InterpretableAI\SystemImage\SysImgBuilder\.julia\packages\IAIBase\qNs6I\src\api.jl:96
  fit!(!Matched::IAIBase.GridSearch{<:IAIBase.SupervisedLearner{<:IAIBase.PolicyTask}}, ::Union{AbstractDataFrame, AbstractMatrix{<:Real}}, ::Union{AbstractDataFrame, AbstractMatrix{<:Real}}, !Matched::Union{AbstractDataFrame, AbstractMatrix{<:Real}}, !Matched::Union{AbstractDataFrame, AbstractMatrix{<:Real}}; sample_weight, kwargs...) at C:\Users\iai\builds\InterpretableAI\SystemImage\SysImgBuilder\.julia\packages\IAIBase\qNs6I\src\api.jl:56
  fit!(!Matched::IAIBase.GridSearch{<:IAIBase.UnsupervisedLearner{<:IAIBase.Im

In [44]:
grid <- IAI.GridSearch(
    IAI.OptimalFeatureSelectionClassifier(
        random_seed = 42,
    ),
    sparsity = 1:10,
)
fit(grid, X_train, y_train, validation_criterion = "auc")

MethodError: MethodError: no method matching -(::IAIBase.GridSearch{OptimalFeatureSelection.OptimalFeatureSelectionClassifier, IAIBase.NullGridResult, IAIBase.Data{IAIBase.ClassificationTask, IAIBase.ClassificationTarget}})
Closest candidates are:
  -(::Any, !Matched::ChainRulesCore.AbstractThunk) at C:\Users\iai\builds\InterpretableAI\SystemImage\SysImgBuilder\.julia\packages\ChainRulesCore\C73ay\src\tangent_types\thunks.jl:35
  -(::T, !Matched::Union{MathOptInterface.ScalarAffineFunction{T}, MathOptInterface.ScalarQuadraticFunction{T}}) where T at C:\Users\iai\builds\InterpretableAI\SystemImage\SysImgBuilder\.julia\packages\MathOptInterface\Ohzb2\src\Utilities\functions.jl:1812
  -(::T1, !Matched::DataValues.DataValue{T2}) where {T1, T2} at C:\Users\shann\.julia\packages\DataValues\N7oeL\src\scalar\operations.jl:75
  ...